In [52]:
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt
from numpy import ma
import pandas as pd
import os
import sklearn
import warnings
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator


# pytorch imports1
import torch
from torch.utils.data.sampler import SequentialSampler, Sampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import preprocessor 
from importlib import reload
reload(preprocessor)

<module 'preprocessor' from 'C:\\Users\\Alec\\OneDrive\\Documents\\Syracuse\\IST707-DataAnalytics\\project\\preprocessor.py'>

In [53]:
path2 = Path('C:/Users/Alec/OneDrive/Documents/Syracuse/IST707-DataAnalytics/project/.data/bejing_air_quality/cleaned')
cleaned_files = list(path2.glob("*.csv"))
lens = []
for file in cleaned_files:
    df = pd.read_csv(file)
    lens.append(len(df))
    
print(lens)

[35064, 35064, 35064, 35064, 35064, 35064, 35064, 35064, 35064, 35064, 35064, 35064]


In [54]:
df = pd.read_csv(cleaned_files[0])

In [50]:
df.head()

,timestamp,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,2013-03-01 00:00:00,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2013-03-01 01:00:00,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,2013-03-01 02:00:00,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,2013-03-01 03:00:00,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,2013-03-01 04:00:00,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  35064 non-null  object 
 1   PM2.5      35064 non-null  float64
 2   PM10       35064 non-null  float64
 3   SO2        35064 non-null  float64
 4   NO2        35064 non-null  float64
 5   CO         35064 non-null  float64
 6   O3         35064 non-null  float64
 7   TEMP       35064 non-null  float64
 8   PRES       35064 non-null  float64
 9   DEWP       35064 non-null  float64
 10  RAIN       35064 non-null  float64
 11  wd         35064 non-null  object 
 12  WSPM       35064 non-null  float64
 13  station    35064 non-null  object 
dtypes: float64(11), object(3)
memory usage: 3.7+ MB


In [11]:
date_time = pd.to_datetime(df.pop('timestamp'), format='%Y-%m-%d %H:%M:%S')

In [12]:
date_time

0       2013-03-01 00:00:00
1       2013-03-01 01:00:00
2       2013-03-01 02:00:00
3       2013-03-01 03:00:00
4       2013-03-01 04:00:00
                ...        
35059   2017-02-28 19:00:00
35060   2017-02-28 20:00:00
35061   2017-02-28 21:00:00
35062   2017-02-28 22:00:00
35063   2017-02-28 23:00:00
Name: timestamp, Length: 35064, dtype: datetime64[ns]

In [14]:
import datetime
timestamp_s = date_time.map(datetime.datetime.timestamp)

In [15]:
timestamp_s

0        1.362114e+09
1        1.362118e+09
2        1.362121e+09
3        1.362125e+09
4        1.362128e+09
             ...     
35059    1.488326e+09
35060    1.488330e+09
35061    1.488334e+09
35062    1.488337e+09
35063    1.488341e+09
Name: timestamp, Length: 35064, dtype: float64

In [16]:
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [17]:
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station,Day sin,Day cos,Year sin,Year cos
0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,0.965926,2.588190e-01,0.856472,0.516193
1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,1.000000,4.371187e-12,0.856842,0.515579
2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,0.965926,-2.588190e-01,0.857211,0.514965
3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,0.866025,-5.000000e-01,0.857580,0.514350
4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,0.707107,-7.071068e-01,0.857949,0.513735


In [18]:
df.columns

Index(['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP',
       'RAIN', 'wd', 'WSPM', 'station', 'Day sin', 'Day cos', 'Year sin',
       'Year cos'],
      dtype='object')

In [58]:
targets = ["PM2.5"]
num_features = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
time_cols = ["timestamp"]
cat_features = ["wd"]

In [72]:
class TimeCosSin(TransformerMixin ):
     #Return self nothing else to do here
     
    @property
    def date_time(self):
        return pd.to_datetime(self.X.pop('timestamp'), format='%Y-%m-%d %H:%M:%S') 
            
    @property 
    def timestamps(self):
        return self.date_time.map(datetime.datetime.timestamp)
        
    @property
    def feature_names(self):
        return ['Day sin', 'Day cos', 'Year sin', 'Year cos']
        
    def fit( self, X, y = None  ):
        return self
    
    def transform(self, X, y=None):
        self.X = X
        day = 24*60*60 
        year = (365.2425)*day
        
        timestamp_s = self.timestamps
        X['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        X['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        X['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        X['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        
        return X.values
        
    def get_feature_names(self, input_features=None):
        return self.feature_names
        

In [73]:
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
]
    
)
time_transformer = Pipeline(steps=[
    ('cos_sin', TimeCosSin())
])
cat_transformer = OneHotEncoder(handle_unknown='ignore')

data_t = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("time", time_transformer, time_cols),
        ("cat", cat_transformer, cat_features),
        
    ]

)

In [74]:
new_df = data_t.fit_transform(df)

In [75]:
data_2 = ColumnTransformer(
    transformers=[
        ("time", time_transformer, time_cols)
        
    ]

)

In [76]:
data_2.fit_transform(df)

array([[ 9.65925826e-01,  2.58819045e-01,  8.56472298e-01,
         5.16192990e-01],
       [ 1.00000000e+00,  4.37118700e-12,  8.56842076e-01,
         5.15578954e-01],
       [ 9.65925826e-01, -2.58819045e-01,  8.57211413e-01,
         5.14964652e-01],
       ...,
       [ 5.00000000e-01,  8.66025404e-01,  8.55627559e-01,
         5.17592002e-01],
       [ 7.07106781e-01,  7.07106781e-01,  8.55998340e-01,
         5.16978571e-01],
       [ 8.66025404e-01,  5.00000000e-01,  8.56368681e-01,
         5.16364873e-01]])

In [81]:
pd.DataFrame(new_df, columns=get_feature_names(data_t))

C:\Users\Alec\anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available


,num__PM10,num__SO2,num__NO2,num__CO,num__O3,num__TEMP,num__PRES,num__DEWP,num__RAIN,num__WSPM,...,cat__x0_NNW,cat__x0_NW,cat__x0_S,cat__x0_SE,cat__x0_SSE,cat__x0_SSW,cat__x0_SW,cat__x0_W,cat__x0_WNW,cat__x0_WSW
0,-1.119286,-0.594827,-1.413086,-0.778654,0.378804,-1.252734,1.071460,-1.601139,-0.074064,2.235205,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.077333,-0.594827,-1.413086,-0.778654,0.378804,-1.287820,1.090682,-1.557313,-0.074064,2.484343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.087822,-0.550998,-1.332014,-0.778654,0.309328,-1.287820,1.119515,-1.557313,-0.074064,3.231757,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.098310,-0.288024,-1.304990,-0.778654,0.291959,-1.314135,1.215625,-1.644965,-0.074064,1.155607,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.129775,-0.244194,-1.277965,-0.778654,0.291959,-1.366763,1.282902,-1.652270,-0.074064,0.242100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-0.857080,-0.550998,-0.656412,-0.698129,0.691445,-0.094900,0.158416,-1.411225,-0.074064,0.574284,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35060,-0.773174,-0.463340,-0.386171,-0.617603,0.448280,-0.173843,0.168027,-1.330877,-0.074064,-0.671406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
35061,-0.773174,-0.331853,0.181334,-0.456553,0.048794,-0.244015,0.225693,-1.199398,-0.074064,-0.505314,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35062,-0.699756,-0.244194,0.748840,-0.456553,-0.350691,-0.270329,0.244915,-1.170181,-0.074064,-0.422268,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [77]:
get_feature_names(data_2)

['cos_sin__Day sin',
 'cos_sin__Day cos',
 'cos_sin__Year sin',
 'cos_sin__Year cos']

In [78]:
get_feature_names(data_t)

C:\Users\Alec\anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available


['num__PM10',
 'num__SO2',
 'num__NO2',
 'num__CO',
 'num__O3',
 'num__TEMP',
 'num__PRES',
 'num__DEWP',
 'num__RAIN',
 'num__WSPM',
 'cos_sin__Day sin',
 'cos_sin__Day cos',
 'cos_sin__Year sin',
 'cos_sin__Year cos',
 'cat__x0_E',
 'cat__x0_ENE',
 'cat__x0_ESE',
 'cat__x0_N',
 'cat__x0_NE',
 'cat__x0_NNE',
 'cat__x0_NNW',
 'cat__x0_NW',
 'cat__x0_S',
 'cat__x0_SE',
 'cat__x0_SSE',
 'cat__x0_SSW',
 'cat__x0_SW',
 'cat__x0_W',
 'cat__x0_WNW',
 'cat__x0_WSW']